In [20]:
val path = "/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
// note: use sbt assembly to create jar
interp.load.cp(os.Path(path))
import $ivy.`org.plotly-scala::plotly-almond:0.8.2`
import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import ujson.Value
import sys.process._

// interp.load.ivy("com.github.pathikrit" %% "better-files" % "3.9.1")
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import better.files._
import $ivy.`com.lihaoyi::ujson:1.3.12`
import $ivy.`com.lihaoyi::requests:0.6.5`



path: String = "/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import $ivy.$                                      

import $ivy.$                             
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`


import ujson.Value

import sys.process._

// interp.load.ivy("com.github.pathikrit" %% "better-files" % "3.9.1")

import $ivy.$                                         

import better.files._

import $ivy.$                          

import $ivy.$                            



In [24]:
import scala.util.Random
import edu.colorado.plv.bounder.Driver
import edu.colorado.plv.bounder.RunConfig
import edu.colorado.plv.bounder.symbolicexecutor.state.PureConstraint
import upickle.default.read
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}

import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._
import upickle.default.{read, write}

import scala.util.Random

import edu.colorado.plv.bounder.Driver

import edu.colorado.plv.bounder.RunConfig

import edu.colorado.plv.bounder.symbolicexecutor.state.PureConstraint

import upickle.default.read

import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}


import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond._

import upickle.default.{read, write}

In [22]:
import edu.colorado.plv.bounder.symbolicexecutor.state.DBOutputMode
import edu.colorado.plv.bounder.symbolicexecutor.state.{DBPathNode,IPathNode}
import edu.colorado.plv.bounder.ir._
import edu.colorado.plv.bounder.symbolicexecutor.QueryFinished
import edu.colorado.plv.bounder.BounderUtil
def obsMessagesStr(nodes:List[DBPathNode]):List[DBPathNode] = {
nodes.flatMap{n =>
  n.qry.loc match {
    case c @ CallinMethodReturn(fmwClazz, fmwName) => Some(n)
    case c @ CallinMethodInvoke(fmwClazz, fmwName) => Some(n)
    case c @ GroupedCallinMethodInvoke(targetClasses, fmwName) => Some(n)
    case c @ GroupedCallinMethodReturn(targetClasses, fmwName) => Some(n)
    case c @ CallbackMethodInvoke(fmwClazz, fmwName, loc) => Some(n)
    case c @ CallbackMethodReturn(fmwClazz, fmwName, loc, line) => Some(n)
    case _ => None
  }}
}

// implicit val db = DBOutputMode("/Users/shawnmeier/Desktop/db_triage/connectbot1.db",true)
implicit val db = DBOutputMode("/Users/shawnmeier/Desktop/bounder_debug_data/deref2.db",true)

import edu.colorado.plv.bounder.symbolicexecutor.state.DBOutputMode

import edu.colorado.plv.bounder.symbolicexecutor.state.{DBPathNode,IPathNode}

import edu.colorado.plv.bounder.ir._

import edu.colorado.plv.bounder.symbolicexecutor.QueryFinished

import edu.colorado.plv.bounder.BounderUtil

defined function obsMessagesStr
db: DBOutputMode = DBOutputMode(
  dbfile = "/Users/shawnmeier/Desktop/bounder_debug_data/deref2.db",
  truncate = true
)

In [36]:
object LiveNodes{ // Create object to supress output TODO: upgrade ammonite at some point
    val live = db.getAllLiveNodes()
    val (xLive,yLive) = live.groupBy(_.depth).map(a => (a._1, a._2.size)).toList.sortBy(_._1).unzip
    val allNodes = db.getAllNodes()
    val (xAll,yAll) = allNodes.groupBy(_.depth).map(a => (a._1, a._2.size)).toList.sortBy(_._1).unzip


    val data = Seq(
        Scatter(xLive, yLive, name="live nodes"),
        Scatter(xAll,yAll, name="all nodes")
    )
}

plot(LiveNodes.data)

defined object LiveNodes
res35_1: String = "plot-ef5a81ef-93fd-47b2-8b7d-a400f5678ae3"

In [30]:
object AvgPureSize{
    val (x,y) = LiveNodes.live.groupBy(_.depth).map{case (depth, nodes) => 
        val pureSizes = nodes.map(n => n.qry.getState.get.sf.pureFormula.size)
        (depth,pureSizes.sum/nodes.size.toDouble)
    }.toList.sortBy(_._1).unzip

}
Scatter(AvgPureSize.x, AvgPureSize.y).plot()

defined object AvgPureSize
res29_1: String = "plot-4ec76283-6f47-46e0-a991-4e9a6a7490c2"

In [33]:
LiveNodes.live.groupBy(_.depth)(160).head.qry.getState.get.toString 

res32: String = "( locals: r0 -> p-8,$r1 -> p-4         heap: p-9.this$0->p-10 * p-2.this$0->p-3 * p-4.this$0->p-5 * p-8.r->p-7 * p-1.r->p-2 * p-7.this$0->p-1 * p-8.r2->p-9 * p-6.r->p-4 * p-6.r2->p-7          pure: p-4  !=  NULL && p-6  !=  p-8 && p-2  !=  p-7 && p-7  !=  NULL && p-2  !=  p-9 && p-5  !=  p-1 && p-2  !=  NULL && p-3  !=  p-1 && p-6  !=  NULL && p-9  !=  NULL && p-1  !=  NULL && p-7  !=  p-9 && p-1  !=  p-8         types: List(p-1:{4114}, p-3:{4114}, p-4:{4121}, p-2:{4120}, p-10:{4114}, p-6:{4114}, p-8:{4114}, p-9:{4120}, p-7:{4120}, p-5:{4114})    trace: (Map() - |> ))"

In [34]:
LiveNodes.live.size

res33: Int = 26506

In [ ]:
// LiveNodes.live.filter(pn => pn.qry.getState.get.sf.traceAbstraction.rightOfArrow.size > 0)
// LiveNodes.live.head
def groupByLocation(nodes:Iterable[DBPathNode]) = {
    
}
groupByLocation(LiveNodes.live)

In [19]:
// Count pure vars in state


Histogram(x=List(1,1,3,5,5,5)).plot()//.withXbins(new Bins(0,1,1)).plot()

res18: String = "plot-54786f8c-00ff-42ed-ba81-b2aea62694ef"

In [ ]:
def getEndLive()= {
    // Note: this cant distinguish live at end from nodes with no pred locations
    val live = db.getLive()
    val filtLive:Set[IPathNode] = live.filter{a => a.qry.isLive && a.subsumed.isEmpty}.map(_.asInstanceOf[IPathNode])
    // BounderUtil.computeDepthOfWitOrLive(filtLive, QueryFinished)
    filtLive
}

def dumpFrom(liveNodes: Set[IPathNode],outF:String):Unit = {
    import edu.colorado.plv.bounder.symbolicexecutor.state.PrettyPrinting
    val pp = new PrettyPrinting()
    val inspQ: Set[IPathNode] = liveNodes
    val pptr = pp.witnessToTrace(inspQ.toList,false)
    println(s"inspection set size: ${inspQ.size}")
    pp.dumpDebugInfo(inspQ.toSet, "timeout", true, Some(outF))
}

dumpFrom(getEndLive(), "/Users/shawnmeier/Desktop/55select/org.andstatus/sample/res_1763/specout/org.andstatus/sample")

In [ ]:
// LiveNodes.live.map{_.depth}
// dumpFrom(LiveNodes.live,"/Users/shawnmeier/Desktop/AntennapodExecute_bug_workingdir")

In [ ]:
def dumpFromDepth(liveNodes: Set[DBPathNode],outF:String, sampDepth:Int):Unit = {
    import edu.colorado.plv.bounder.symbolicexecutor.state.PrettyPrinting
    val pp = new PrettyPrinting()
    val inspQ: Set[IPathNode] = liveNodes.filter(n => n.depth == sampDepth).map(_.asInstanceOf[IPathNode])
    val pptr = pp.witnessToTrace(inspQ.toList,false)
    println(s"inspection set size: ${inspQ.size}")
    pp.dumpDebugInfo(inspQ.toSet, "timeout", true, Some(outF))
}
// pp.qryString(inspQ.qry)
// Test1.pptr.take(3)

dumpFromDepth(LiveNodes.live, 
              "/Users/shawnmeier/Desktop/AntennapodExecute_fix_workingdir", 
                38)

In [ ]:
def getEXSet() = {
    val locGroup = LiveNodes.live.groupBy(n => n.qry.loc)
    val mostVisitedList = locGroup.map{a => (a._1,a._2.size)}.toList.sortBy(a => a._2).reverse
    val mostVisitedNode = mostVisitedList.head._1
    
    val pp = new PrettyPrinting()
    val nodeList = locGroup(mostVisitedNode)
    val outf = File("/Users/shawnmeier/Desktop/db_triage/states.txt")
    nodeList.foreach{n => 
        outf.appendLine().append(pp.qryString(n.qry))
        outf.appendLine().append(s"subs: ${n.subsumedID}")
    }
}
    
getEXSet()

In [ ]:
LiveNodes.live.groupBy(n => n.subsumed.isDefined).map{a => (a._1,a._2.size)}

In [ ]:
LiveNodes.live.groupBy(_.dbgTrace).map(a => (a._1.toString,a._2.size)).toList.sortBy(_._2).reverse

In [ ]:
LiveNodes.live.filter(_.dbgTrace.toString == "Some(List(I_CBEnter_Activity_onCreate, I_CIExit_Activity_findView, FreshRef(LS__10335), I_CIExit_View_setOnClickListener, I_CBEnter_Activity_onResume, I_CIExit_Activity_finish, I_CIExit_View_setOnClickListener, I_CBEnter_ViewOnClickListener_onClick))").groupBy(_.dbgHeap).map(a => (a._1.toString,a._2.size)).toList.sortBy(_._2).reverse.map(println)

In [ ]:
object OUT32{
    import edu.colorado.plv.bounder.symbolicexecutor.state.State
    val dbgNodes = LiveNodes.live.filter(_.dbgTrace.toString == "Some(List(I_CIExit_Dialog_show, I_CIEnter_Dialog_dismiss))").groupBy(_.dbgHeap).map(a => (a._1.toString,a._2.size,a._2)).toList.sortBy(_._2).reverse
    val nodeList = OUT32.dbgNodes.head._3.toList
    def pp(s:State) = {
        val o = List("locals:","heap:","types:","trace:").foldLeft(s.toString){
            case (acc,v) => acc.replace(s"$v", s"\n$v")
        }
        println(o)
    }
    val stateList = nodeList.map(n => n.qry.getState.get)
    val stackGrouped = stateList.groupBy(_.sf.callStack.toString()).toList.sortBy(_._2.size).reverse
}
OUT32.stackGrouped.map(a => (a._1, a._2.size))
//OUT32.stateList.sortBy(s => s.sf.callStack.toString()).take(10).foreach{OUT32.pp}


In [ ]:
import upickle.default.{macroRW, read, write, ReadWriter => RW}
val js = ujson.Arr(List(List(1),List(2)))
js.toString

In [ ]:
List("fooo","bar","a").sortBy(_.length).take(5)